In [1]:
from hdimvis.metrics.stress.stress import vectorised_stress,unvectorised_stress
from hdimvis.data_fetchers.DataFetcher import DataFetcher
from hdimvis.metrics.distance_measures.euclidian_and_manhattan import euclidean,manhattan
import numpy as np
import tracemalloc
from time import perf_counter
import definitions
import os
import matplotlib.pylab as plt

In [2]:
large_dataset = DataFetcher.fetch_data('mnist', size='max')


####################
Fetching the "mnist" dataset


C:\Users\Owner\.virtualenvs\2095736w-0SnFieZ0\lib\site-packages\sklearn\datasets\_openml.py:932: FutureWarning: The default value of `parser` will change from `'liac-arff'` to `'auto'` in 1.4. You can set `parser='auto'` to silence this warning. Therefore, an `ImportError` will be raised from 1.4 if the dataset is dense and pandas is not installed. Note that the pandas parser may return different data types. See the Notes Section in fetch_openml's API doc for details.
  warn(


####################
Dataset loaded
Dataset shape: (70000, 784)
####################


In [ ]:
sizes = np.arange(10,2000, 20)
print(sizes)
sizes = [3000]
num_repeats = 6

# 3 for memory use base (index 0), memory use peak (1) and time (index 2),
vectorised_data = np.zeros((len(sizes),3,num_repeats))
un_vectorised_data = np.zeros((len(sizes),3,num_repeats))

collected_data = [vectorised_data, un_vectorised_data]
functions = [vectorised_stress, unvectorised_stress]

In [ ]:
for i, size in enumerate(sizes):
    for j in range(2):
        for k in range(num_repeats):
            print(size)
            sample_indices = np.random.randint(0, 69999, size)
            sample = large_dataset.data[sample_indices]
            ld_positions = 20*np.random.rand(sample.shape[0],2)
            stress_fn = functions[j]

            tracemalloc.start()
            stress1 = stress_fn(sample, ld_positions, euclidean)
            collected_data[j][i,0,k] = tracemalloc.get_traced_memory()[0]
            collected_data[j][i,1,k] = tracemalloc.get_traced_memory()[1]
            tracemalloc.stop()

            start = perf_counter()
            stress2 = stress_fn(sample, ld_positions, euclidean)
            collected_data[j][i,2,k] = perf_counter() - start



2000


In [10]:
output_dir = os.path.realpath(os.path.join(definitions.PROJECT_ROOT, "experiments/stress_vectorisation/out/stress_vec_1.npy"))

with open(output_dir, 'wb') as f:
    np.save(f, vectorised_data)
    np.save(f, un_vectorised_data)

In [ ]:
size = 3000
sample_indices = np.random.randint(0, 69999, size)
sample = large_dataset.data[sample_indices]
ld_positions = 20*np.random.rand(sample.shape[0],2)
stress_ts = vectorised_stress(sample, ld_positions, euclidean)

